# Cotton Fiber Object Detection

Jeremy Park

jipark@ncsu.edu

This notebook contains all of the code for cotton fiber tapered-to-hemispherical ratio computer vision predictions.

From the downloading of the datasets from Roboflow to the predictions on cotton breeds, this Google Colab notebook handles it all.

#### Detectron2 blogs used as inspiration.

I used these two blogs to help write the Detectron2 code portions:

https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5

https://towardsdatascience.com/object-detection-in-6-steps-using-detectron2-705b92575578

# On Google Drive, mount to your local drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dependencies

### Install PyTorch

In [ ]:
!pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

### Install Detectron2

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

### Install Libraries

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetMapper, MetadataCatalog, DatasetCatalog, build_detection_test_loader, build_detection_train_loader
import os
import cv2
import itertools
import copy
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
import skimage
from skimage.transform import rescale
from detectron2.data.datasets import register_coco_instances
import logging
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import time
import datetime
from datetime import datetime
from detectron2.data.transforms import ResizeShortestEdge
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from collections import Counter
import json
!pip install roboflow
from roboflow import Roboflow
import torch
import gc
import pandas as pd
import seaborn as sns
from math import e
from math import ceil
from scipy.stats import wasserstein_distance, entropy
from sklearn.metrics import mutual_info_score
from scipy.special import rel_entr

# Variables

In [ ]:
MY_DRIVE = '/content/drive/My Drive'
HOME = MY_DRIVE + '/Cotton_Project'
MODEL_NAME = "/m1_69_images_human_AI_3_27_22"
OUTPUT_NAME = "/Final Product"
DATA_PATH = HOME + "/Gh_data" + MODEL_NAME
OUTPUT_DIR = HOME + OUTPUT_NAME
os.makedirs(DATA_PATH, exist_ok=True)

print(DATA_PATH)
print(OUTPUT_DIR)

%cd $DATA_PATH
%pwd
%ls

VISUALIZE_TRAIN = False
VISUALIZE_KEYENCE = False
KEYENCE_RATIO = False
RESUME_TRAINING = False
TRAINING = False
EVALUATE_MODEL = True
ROBOFLOW = False
TEST_THRESHOLD = 0.7
TRAINING_VALIDATION_CURVE = True
DISPLAY_BARCHARTS = False
DISPLAY_PERCENTAGES = True
CALCULATE_RATIOS = False
today = datetime.today().strftime('%Y-%m-%d')

# Setting up the data

### Set up train, val, test folders

In [ ]:
%cd $DATA_PATH
%pwd
%ls

# printing the number of images in each folder
%cd $DATA_PATH
%cd './train'
%ls -l . | egrep -c '^-'

%cd $DATA_PATH
%cd './valid'
%ls -l . | egrep -c '^-'

%cd $DATA_PATH
%cd './test'
%ls -l . | egrep -c '^-'

%cd $DATA_PATH

### Register each dataset in Detectron2 COCO format.

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
register_coco_instances("train", {}, DATA_PATH + "/train/_annotations.coco.json", DATA_PATH + "/train")
register_coco_instances("valid", {}, DATA_PATH + "/valid/_annotations.coco.json", DATA_PATH + "/valid")
register_coco_instances("test", {}, DATA_PATH + "/test/_annotations.coco.json", DATA_PATH + "/test")

### Visualize example images and annotations in training

In [ ]:
dataset_dicts = DatasetCatalog.get("train")
train_metadata = MetadataCatalog.get("train")
VISUALIZE_TRAIN=False
if VISUALIZE_TRAIN:
  for d in random.sample(dataset_dicts, 3):
      print(d["file_name"])
      img = cv2.imread(d["file_name"])
      visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
      vis = visualizer.draw_dataset_dict(d)
      cv2_imshow(vis.get_image()[:, :, ::-1])

# Hooks and Custom Detectron2 trainer

Hooks are called at each validation step. So we will use hooks to calculate validation AP.

The custom trainer is used to turn off default augmentations inherent to Detectron2.

In [ ]:
# validation loss code from the help of:
# https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b
class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()
        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

# https://github.com/facebookresearch/detectron2/issues/2114
class BestCheckpointer(HookBase):
    def before_train(self):
        self.best_metric = 0.0
        self.logger = logging.getLogger("detectron2.trainer")
        self.logger.info("######## Running best check pointer")

    def after_step(self):
        metric_name = 'bbox/AP50'
        if metric_name in self.trainer.storage._history:
            eval_metric, batches = self.trainer.storage.history(metric_name)._data[-1]
            if self.best_metric < eval_metric:
                # add 1 to get to the whole number, just for output purposes
                # since it starts at index 0 with iteration 0
                new_batch_number = int(batches) + 1
                eval_metric = str(round(eval_metric, 2))
                if batches % 100 == 0:
                  self.logger.info("############# New best metric: {}".format(eval_metric))
                self.trainer.checkpointer.save("model_{}_iter_{}_with_AP_{:.2f}".format(today, new_batch_number,eval_metric))

# creating our own trainer class to add validation hooks and turn off default augmentations in Detectron2.
class CocoTrainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
      os.makedirs("coco_eval", exist_ok=True)
      output_folder = "coco_eval"
    return COCOEvaluator(dataset_name, cfg, False, output_folder)
  # turning off default augmentations  
  def build_hooks(self):
    hooks = super().build_hooks()
    hooks.insert(-1,LossEvalHook(
        cfg.TEST.EVAL_PERIOD,
        self.model,
        build_detection_test_loader(
            self.cfg,
            self.cfg.DATASETS.TEST[0],
            DatasetMapper(self.cfg,True,augmentations=[])
        )
    ))
    hooks.append(BestCheckpointer())
    return hooks

  @classmethod
  def build_train_loader(cls, cfg):
      mapper = DatasetMapper(cfg, is_train=True, augmentations=[])
      return build_detection_train_loader(cfg, mapper=mapper)

#### Garbage collection to free up RAM, in theory.

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Train!

Perform transfer learning from Faster RCNN R50 FPN 3x model weights, downloaded from Detectron2's model zoo.

In [ ]:
# help from https://towardsdatascience.com/how-to-train-detectron2-on-custom-object-detection-data-be9d1c233e4
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("valid",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0125
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.TEST.EVAL_PERIOD = 100
cfg.OUTPUT_DIR = OUTPUT_DIR
cfg.INPUT.MIN_SIZE_TEST = 1440
cfg.INPUT.MAX_SIZE_TEST = 2000
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=RESUME_TRAINING)
if TRAINING:
  trainer.train()

# Training / Validation Curve 

In [ ]:
# https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b
def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines
if TRAINING_VALIDATION_CURVE:
  experiment_metrics = load_json_arr(OUTPUT_DIR + '/metrics.json')
  plt.plot(
      [x['iteration'] for x in experiment_metrics if 'total_loss' in x], 
      [x['total_loss'] for x in experiment_metrics if 'total_loss' in x])
  plt.plot(
      [x['iteration'] for x in experiment_metrics if 'validation_loss' in x], 
      [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x])
  plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
  plt.xlabel('Iteration', fontsize=14)
  plt.ylabel('Loss', fontsize=14)
  plt.show()

# Evaluation

#### Function to get the model paths from a given output directory

In [ ]:
def get_model_paths(output_directory):
  print("Current output directory: " + output_directory)
  file_paths = os.listdir(output_directory) 
  model_paths = [file_name for file_name in file_paths if "model_" in file_name] 
  return model_paths

#### Get the best model evaluated on a given dataset from a list of model paths

In [ ]:
def instantiate_model(cfg, model_path, dataset_name, output_dir):
  # set the model weights
  cfg.MODEL.WEIGHTS = os.path.join(output_dir, model_path)
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = TEST_THRESHOLD
  # prepare the dataset 
  print("dataset_name: " + dataset_name)
  print(cfg.MODEL.WEIGHTS)
  cfg.DATASETS.TEST = (dataset_name, )
  dataset_dicts = DatasetCatalog.get(dataset_name)
  val_metadata = MetadataCatalog.get(dataset_name)
  # create the predictor
  predictor = DefaultPredictor(cfg)
  # visualize some predictions
  if VISUALIZE_KEYENCE:
    for d in random.sample(dataset_dicts, 3): 
        print(d["file_name"])   
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                      metadata=val_metadata
        )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2_imshow(v.get_image()[:, :, ::-1])

        visualizer = Visualizer(im[:, :, ::-1], metadata=val_metadata)
        vis = visualizer.draw_dataset_dict(d)
        cv2_imshow(vis.get_image()[:, :, ::-1])

  return cfg, predictor, dataset_dicts, val_metadata

### Call COCO Evaluation to get Average Precision scores


In [ ]:
def evaluate_model(cfg, predictor, dataset_name):
  evaluator = COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR)
  val_loader = build_detection_test_loader(cfg, dataset_name, mapper = DatasetMapper(cfg, is_train=False, augmentations=[]))
  print(inference_on_dataset(predictor.model, val_loader, evaluator))

### Calculate T and H ratios with standard deviation for Detectron2 COCO formatted datasets

Assuming these datasets were for just one breed: DP90.

In [ ]:
def calculate_ratios(predictor, dataset_dicts, val_metadata, test_set_name):
  # initialize the count variables
  prediction_t = 0
  prediction_h = 0
  ground_t = 0
  ground_h = 0
  SANITY_CHECK = False
  pred_df = []
  ground_df = []
  # images are weighed proportional to how much they contribute to the total count of fibers
  # for example, an image with 98 fibers should be weighed more in the grand total %T proportion
  # than an image with just 2 fibers.
  pred_weights = []
  ground_weights = []
  eval_column_names = ["Genotype", "Image Name", "Human Total", "ML Total", "Human # T", "ML # T", "Human # H", "ML # H", "Human % T", "ML % T"]
  eval_df = pd.DataFrame(columns = eval_column_names)
  if KEYENCE_RATIO:
    for d in dataset_dicts: 
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        # https://github.com/facebookresearch/detectron2/issues/147
        prediction_list = outputs['instances'].pred_classes.flatten().tolist()
        prediction_counter = Counter(prediction_list)
        prediction_h += prediction_counter[1]
        prediction_t += prediction_counter[2]
        ground_truth_classes = [annotation['category_id'] for annotation in d['annotations']]
        ground_truth_counter = Counter(ground_truth_classes)
        ground_h += ground_truth_counter[1]
        ground_t += ground_truth_counter[2]
        current_pred_ratio_sum = prediction_counter[1] + prediction_counter[2]
        try:
          current_tapered_percentage = round(prediction_counter[2] / current_pred_ratio_sum, 2)
        except:
          # in the case where there are no predictions
          current_tapered_percentage = 0
        pred_df.append(current_tapered_percentage)
        current_ground_ratio_sum = ground_truth_counter[1] + ground_truth_counter[2]
        current_ground_t_percentage = round(ground_truth_counter[2] / current_ground_ratio_sum, 2)
        ground_df.append(current_ground_t_percentage)
        pred_weights.append(prediction_counter[2])
        ground_weights.append(ground_truth_counter[2])
        image_path = d["file_name"]
        image_name = image_path.rsplit('/', 1)[-1]
        image_name = image_name[:-4]
        print("Image Name:" + image_name)
        eval_data = {"Genotype": [test_set_name],
                     "Image Name": [image_name],
                     "Human Total": [current_ground_ratio_sum],
                     "ML Total": [current_pred_ratio_sum],
                     "Human # T":[ground_truth_counter[2]],
                     "ML # T":[prediction_counter[2]],
                     "Human # H":[ground_truth_counter[1]],
                     "ML # H":[prediction_counter[1]],
                     "Human % T":[current_ground_t_percentage],
                     "ML % T":[current_tapered_percentage]}

        eval_data = pd.DataFrame(eval_data)
        eval_frames = [eval_df, eval_data]
        eval_df = pd.concat(eval_frames)
        display(eval_df)

        if SANITY_CHECK:
          print("prediction_h: " + str(prediction_counter[1]))
          print("prediction_t: " + str(prediction_counter[2]))
          print("ground_h: " + str(ground_truth_counter[1]))
          print("ground_t: " + str(ground_truth_counter[2]))
          annotations = [annotation['bbox'] for annotation in d['annotations']]
          hemispherical_annotations = [annotation['bbox'] for annotation in d['annotations'] if annotation['category_id'] == 1]
          tapered_annotations = [annotation['bbox'] for annotation in d['annotations'] if annotation['category_id'] == 2]
          # convert to xyxy format
          for i in range(len(annotations)):
            annotations[i] = BoxMode.convert(annotations[i], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
          for i in range(len(hemispherical_annotations)):
            hemispherical_annotations[i] = BoxMode.convert(hemispherical_annotations[i], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
          for i in range(len(tapered_annotations)):
            tapered_annotations[i] = BoxMode.convert(tapered_annotations[i], BoxMode.XYWH_ABS, BoxMode.XYXY_ABS)
          print("### Ground Truth ###")
          visualizer = Visualizer(im[:, :, ::-1], metadata=val_metadata)
          vis = visualizer.draw_dataset_dict(d)
          cv2_imshow(vis.get_image()[:, :, ::-1])
          print("### Predictions ###")
          v = Visualizer(im[:, :, ::-1],
                        metadata=val_metadata
          )
          v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
          cv2_imshow(v.get_image()[:, :, ::-1])

    prediction_counter = [0,prediction_h, prediction_t]
    ground_truth_counter = [0,ground_h, ground_t]
    groundTruth = [ground_truth_counter[2], ground_truth_counter[1]]
    predictions = [prediction_counter[2], prediction_counter[1]]
    gt_ratio_sum = ground_truth_counter[2] + ground_truth_counter[1]
    try:
      gt_ratio_t = ground_truth_counter[2] / gt_ratio_sum   
      gt_ratio_h = ground_truth_counter[1] / gt_ratio_sum
    except:
      # if DivideByZeroError, set number of fibers to 0
      gt_ratio_t = 0
      gt_ratio_h = 0
    pred_ratio_sum = prediction_counter[2] + prediction_counter[1]
    try:
      pred_ratio_t = prediction_counter[2] / pred_ratio_sum 
      pred_ratio_h = prediction_counter[1] / pred_ratio_sum
    # if DivideByZeroError, set number of fibers to 0
    except:
      pred_ratio_t = 0
      pred_ratio_h = 0
    gt_ratio_t = str(round(gt_ratio_t * 100, 2))
    gt_ratio_h = str(round(gt_ratio_h * 100, 2))
    pred_ratio_t = str(round(pred_ratio_t * 100, 2))
    pred_ratio_h = str(round(pred_ratio_h * 100, 2))
    ratios = [(gt_ratio_t, gt_ratio_h), (pred_ratio_t, pred_ratio_h)]
    assert prediction_h == prediction_counter[1]
    assert prediction_t == prediction_counter[2]
    assert len(pred_weights) == len(ground_weights) == len(dataset_dicts)
    assert sum(ground_weights) == ground_t
    assert sum(pred_weights) == prediction_t
    pred_df = pd.DataFrame(pred_df,columns=['Percent Tapered'])
    ground_df = pd.DataFrame(ground_df,columns=['Percent Tapered'])
    pred_df_list = pred_df['Percent Tapered'].to_list()
    ground_df_list = ground_df['Percent Tapered'].to_list()
    try:
      pred_weights = [round(current_count / prediction_t, 2) for current_count in pred_weights]
    except:
      pred_weights = [0 for current_count in pred_weights]
    try:
      ground_weights = [round(current_count / ground_t, 2) for current_count in ground_weights]
    except:
      ground_weights = [0 for current_count in ground_weights]
    gt_percent_t = round(float(gt_ratio_t) / 100, 3)
    gt_percent_h = round(float(gt_ratio_h) / 100, 3)
    pred_percent_t = round(float(pred_ratio_t) / 100, 3)
    pred_percent_h = round(float(pred_ratio_h) / 100, 3)
    print("Ground Truth T-H percentages: {} - {}".format(gt_percent_t, gt_percent_h))
    print("Prediction T-H ratio: {} - {}".format(pred_percent_t, pred_percent_h))
    # display side by side barcharts comparing ground truth to predictions
    if DISPLAY_BARCHARTS:
      gt_tapered = [groundTruth[0], "Tapered", "Ground Truth"]
      gt_hemispherical = [groundTruth[1], "Hemispherical", "Ground Truth"]
      pred_tapered = [predictions[0], "Tapered", "Prediction"]
      pred_hemispherical = [predictions[1], "Hemispherical", "Prediction"]
      df = pd.DataFrame([gt_tapered, gt_hemispherical, pred_tapered, pred_hemispherical],columns=('Count', 'Class', 'Type'))
      seaborn_plot = sns.catplot(x='Type', y='Count', hue='Class', data=df, kind='bar', palette=sns.color_palette("Pastel1"), color=123, saturation=1)
      seaborn_plot.fig.set_size_inches(10,8)
      seaborn_plot.fig.subplots_adjust(top=0.81,right=0.86) 
      ax = seaborn_plot.facet_axis(0,0)
      count = 0
      percentages = [gt_ratio_t, pred_ratio_t, gt_ratio_h, pred_ratio_h]
      # citation:
      # https://stackoverflow.com/questions/55586912/seaborn-catplot-set-values-over-the-bars
      for p in ax.patches:
          height = int(p.get_height())
          ax.text(p.get_x() + 0.05, 
                  p.get_height() * 1.02, 
                '{} ({}%)'.format(height, percentages[count]), 
                  color='black', 
                  rotation='horizontal',
                  size='large')
          count +=1
      # citation
      # https://stackoverflow.com/questions/29813694/how-to-add-a-title-to-seaborn-facet-plot
      seaborn_plot.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
      seaborn_plot.fig.suptitle("Ground Truth vs. Predictions with {}-Trained Model on {} Image {} Holdout Set".format(train_set_name, len(dataset_dicts), test_set_name))
    # display histograms of the distributions across images.
    if DISPLAY_PERCENTAGES:
      # https://www.geeksforgeeks.org/plotting-histogram-in-python-using-matplotlib/
      fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(16,8))
      axes = plt.gca()
      axs = [ax1, ax2]
      dfs = [ground_df_list, pred_df_list]
      axs[0].set_title("GT % T on {} Image {} Test Set".format(len(dataset_dicts), test_set_name))
      axs[1].set_title("Pred % T on {} Image {} Test Set".format(len(dataset_dicts), test_set_name))
      (ground_hist, bins_gt, patches) = axs[0].hist(dfs[0], bins=20, range=[0, 1], facecolor='#2a4d69', align='mid', weights = ground_weights)
      first_nonzero_index = next((i for i, x in enumerate(ground_hist) if x), None) # x!= 0 for strict match
      if first_nonzero_index == 0:
        first_nonzero_index == first_nonzero_index
      else:
        first_nonzero_index -= 1
      handles, _ = axs[0].get_legend_handles_labels()
      handles.append(axs[0].axvline(gt_percent_t, color='#fe9c8f', linestyle='dashed', linewidth=1))
      axs[0].legend(loc='upper left',handles = handles, labels = ['Total GT % T'])
      (pred_hist, bins_pred, patches) = axs[1].hist(dfs[1], bins=20, range=[0, 1], facecolor='#854442', align='mid', weights = pred_weights)
      first_nonzero_index_pred = next((i for i, x in enumerate(pred_hist) if x), None) # x!= 0 for strict match
      if first_nonzero_index_pred == 0:
        first_nonzero_index_pred == first_nonzero_index_pred
      else:
        first_nonzero_index_pred -= 1
      largest_x = max(bins_gt[first_nonzero_index], bins_pred[first_nonzero_index_pred])
      largest_y = max(max(ground_hist), max(pred_hist))
      largest_y = float(ceil(largest_y*100) / 100)
      ax1.set_xlim([largest_x,1])
      ax2.set_xlim([largest_x,1])
      ax1.set_ylim([0,largest_y])
      ax2.set_ylim([0,largest_y])
      ax1.set_xlabel("Tapered Percentage")
      ax2.set_xlabel("Tapered Percentage")
      ax1.set_ylabel("Proportion")
      ax2.set_ylabel("Proportion")
      fig.tight_layout(pad=3.0)
      handles, _ = axs[1].get_legend_handles_labels()
      handles.append(axs[1].axvline(pred_percent_t, color='#35a79c', linestyle='dashed', linewidth=1))
      axs[1].legend(loc='upper left',handles = handles, labels = ['Total Pred % T'])
      pred_std_dev = round(pred_df.std(numeric_only=True)['Percent Tapered'], 3)
      ground_std_dev = round(ground_df.std(numeric_only=True)['Percent Tapered'], 3)
      emd = round(wasserstein_distance(ground_hist, pred_hist),5)
      print("Prediction Std Dev: {}".format(pred_std_dev))
      print("Ground Std Dev: {}".format(ground_std_dev))
      print("Earth Mover's Distance: {}".format(emd))
      human_ratio = gt_ratio_t + '-' + gt_ratio_h
      ML_ratio = pred_ratio_t + '-' + pred_ratio_h
      ratio_diff = abs(float(gt_ratio_t) - float(pred_ratio_t))
      data = {"Genotype": ["DP90"], "Total Images": len(dataset_dicts), "Human T:H Ratio":[human_ratio], "Human Total Fibers": [gt_ratio_sum],
              "ML T:H Ratio": [ML_ratio], "ML Total Fibers": [pred_ratio_sum], "Ratio Diff": [ratio_diff],
              "Human Std Dev":[round(ground_std_dev*100, 1)], "ML Std Dev":[round(pred_std_dev*100, 1)]}
      keyence_df = pd.DataFrame(data)
    
    return ratios, keyence_df, eval_df

### Calculate Ratio Difference between Ground Truth and Prediction %T

In [ ]:
def calculate_ratio_difference(groundTruth, prediction):
  t_diff = abs(float(groundTruth[0]) - float(prediction[0])) 
  h_diff = abs(float(groundTruth[1]) - float(prediction[1]))
  return t_diff + h_diff

def get_ratio_difference(predictor, dataset_dicts, val_metadata, train_set_name, test_set_name):
  ratios, keyence_df, eval_df = calculate_ratios(predictor, dataset_dicts, val_metadata, test_set_name)
  return calculate_ratio_difference(ratios[0], ratios[1]), keyence_df, eval_df

# Final Evaluation

### Model Evaluation

In [ ]:
def model_evaluation(cfg, model_path, dataset_name, train_set_name, test_set_name, output_dir):
  current_model_ratio_difference = 10000
  cfg, predictor, dataset_dicts, val_metadata = instantiate_model(cfg, model_path, dataset_name, output_dir)
  current_model_ratio_difference, keyence_row, eval_data = get_ratio_difference(predictor, dataset_dicts, val_metadata, train_set_name, test_set_name)
  evaluate_model(cfg, predictor, dataset_name)
  return current_model_ratio_difference, keyence_row, eval_data

# pick model based on smallest ratio difference across all images.
def best_val_evaluation(model_paths, cfg, dataset_name, train_set_name, test_set_name, output_dir):
  best_model_difference = 10000
  best_model_path_name = ''
  print("Current Test Set: " + test_set_name)
  print("Current Dataset Dictionaries: " + dataset_name)
  for model_path in model_paths:
    print("Current Model: " + model_path)
    current_model_ratio_difference, keyence_row = model_evaluation(cfg, model_path, dataset_name, train_set_name, test_set_name, output_dir)
    if current_model_ratio_difference < best_model_difference:
      best_model_difference = current_model_ratio_difference
      best_model_path_name = model_path
      print("Best Model: " + model_path)
      print("Best Model Ratio Difference: {}".format(best_model_difference))
  return best_model_path_name

### Evaluate Keyence-Trained Model on Keyence Holdout Set

In [ ]:
def final_evaluation(cfg, best_model_path_name, train_set_name, test_set_name, output_dir):
  MODEL_PATH_NAME = best_model_path_name.split(".pth",1)[0]
  DATASET_FILENAME = "/DP90_total_results_{}_{}.csv".format(today,MODEL_PATH_NAME)
  SUMMARY_DP90_FILENAME = HOME  + DATASET_FILENAME
  print("Summary results filename: " + SUMMARY_DP90_FILENAME)
  EVAL_DATASET_FILENAME = "/DP90_per_image_results_{}_{}.csv".format(today, MODEL_PATH_NAME)
  INDIVIDUAL_DP90_FILENAME = HOME + EVAL_DATASET_FILENAME
  print("Individual image results filename: " + INDIVIDUAL_DP90_FILENAME)
  column_names = ["Genotype", "Total Images", "Human T:H Ratio", "Human Total Fibers", "ML T:H Ratio", "ML Total Fibers","Ratio Diff", "Human Std Dev", "ML Std Dev"]
  geno_df = pd.DataFrame(columns = column_names)
  eval_column_names = ["Genotype", "Image Name", "Human Total", "ML Total", "Human # T", "ML # T", "Human # H", "ML # H", "Human % T", "ML % T"]
  eval_df = pd.DataFrame(columns = eval_column_names)
  HOLDOUT_SETS = ["valid","test"]

  for holdout_set in HOLDOUT_SETS:
    test_set_name = ''
    if holdout_set is "test":
      test_set_name = "Test"
    elif holdout_set is "valid":
      test_set_name = "Validation"  
   
    diff, keyence_row, eval_data = model_evaluation(cfg, best_model_path_name, holdout_set, train_set_name, test_set_name, output_dir)
    frames = [geno_df, keyence_row]
    geno_df = pd.concat(frames)
    display(geno_df)
    eval_frames = [eval_df, eval_data]
    eval_df = pd.concat(eval_frames)
    display(eval_df)

  geno_df.to_csv(SUMMARY_DP90_FILENAME)
  eval_df.to_csv(INDIVIDUAL_DP90_FILENAME)

### Run Final Evaluation

In [ ]:
DISPLAY_BARCHARTS = False
CALCULATE_RATIOS = True
EVALUATE_MODEL = False
dataset_name = "valid"
train_set_name = "Keyence"
test_set_name = "Keyence"
KEYENCE_RATIO = True
output_dir = OUTPUT_DIR
DISPLAY_BARCHARTS = False
CALCULATE_RATIOS = True
EVALUATE_MODEL = False
best_model_path_name = 'model_weights.pth'
final_evaluation(cfg, best_model_path_name, train_set_name, test_set_name, output_dir)
# in case you want to find the model with the lowest ratio difference.
# best_val_evaluation(model_paths, cfg, dataset_name, train_set_name, test_set_name, output_dir)

# Evaluate DP90 Model on Final Evaluation Sets

### Read in CSV with genotype T:H ratios

In [ ]:
DATASET_FILENAME_DIR = "/Analysis10_Final"
DATASET_FILENAME = "/Cotton Test Accession T_H Data v2.csv"
DP90_CSV = '/DP90 Image Variation.csv'
COKER312_CSV = '/Coker 312 Image 25-48 Variation.csv'
HALF_HALF_CSV = '/Half_Half Image Variation.csv'
TH_DATASET = HOME + DATASET_FILENAME_DIR + DATASET_FILENAME
COLLABORATION_DIR = "/content/drive/.shortcut-targets-by-id/1ycJ7eVD7s1isWTBKLA7SgH1qsKQrmM--/"
th_df = pd.read_csv(TH_DATASET)
dp90_df = pd.read_csv(HOME + DATASET_FILENAME_DIR + DP90_CSV)
coker312_df = pd.read_csv(HOME + DATASET_FILENAME_DIR + COKER312_CSV)
half_half_df = pd.read_csv(HOME + DATASET_FILENAME_DIR + HALF_HALF_CSV)
dp90_df=dp90_df.rename(columns={"Image Nam/ROI coords":"Image Name", 'Image #T':"#T", 'Image #H':"#H", '%T':"%T",'%H':"%H",'Total Fiber':'Total'})
coker312_df=coker312_df.rename(columns={"Image Nam/ROI coords":"Image Name", 'Image #T':"#T", 'Image #H':"#H", '%T':"%T",'%H':"%H",'Total Fiber':'Total'})
half_half_df=half_half_df.rename(columns={"Image Name; ROI coords":"Image Name", 'Image #T':"#T", 'Image #H':"#H", '%T':"%T",'%H':"%H",'Total Fiber':'Total'})
frames = [dp90_df, coker312_df, half_half_df]
greenhouse_df = pd.concat(frames)
th_df = th_df[['Genotype','Drive Path', '%T', '%H', '#T', '#H']]
# things change in the path name.
th_df['Drive Path'] = th_df['Drive Path'].str.replace('>', "/")
th_df['Drive Path'] = th_df['Drive Path'].str.replace('Test Accession Images', "Evaluation Test Accession Images")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gb cv Pima 3-79', "Gb cv Pima 379")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gh cv Half and Half Test Set', "")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gh cv Coker 312 Test Set', "")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gh cv Deltapine 90 Test Set', "")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gb cv Phytogen 800 Test Set', "")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gb cv Pima 379 Test Set', "")
th_df['Drive Path'] =th_df['Drive Path'].str.replace('Gb cv Pima S7 Test Set', "")
th_df['Drive Path'] = th_df['Drive Path'].str.replace('Modeling', "Machine Learning")
display(th_df)
display(greenhouse_df)
%cd $COLLABORATION_DIR

### Create a jpg for an image if no jpg exists

In [ ]:
# citation:
# https://stackoverflow.com/questions/28870504/converting-tiff-to-jpeg-in-python
def create_jpg(genotype_image_dir):
  if any(File.endswith(".jpg") for File in os.listdir(genotype_image_dir)):
      return
  else:
    yourpath = genotype_image_dir
    for root, dirs, files in os.walk(yourpath, topdown=False):
        for name in files:
            print(os.path.join(root, name))
            if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":
                if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                    print("A jpeg file already exists for %s" % name)
                # If a jpeg is *NOT* present, create one from the tiff.
                else:
                    outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                    try:
                        im = Image.open(os.path.join(root, name))
                        print("Generating jpeg for %s" % name)
                        im.thumbnail(im.size)
                        im.save(outfile, "JPEG", quality=100)
                    except:
                        print("FAIL!")

### Function to return all image paths with .jpg ending in a given directory

In [ ]:
def get_image_paths(current_genotype_dir):
  current_genotype_dir = current_genotype_dir.strip()
  create_jpg(current_genotype_dir)
  file_paths = os.listdir(current_genotype_dir)
  image_paths = [current_genotype_dir + "/" + file_name for file_name in file_paths if ".jpg" in file_name] 
  return image_paths

### Calculate the ratios for the evaluation set data.

This code is slightly different than the Detectron2 formatted data, since we are now accepting CSV format.

In [ ]:
def get_genotype_ratios(predictor, row, val_metadata, train_set_name, greenhouse_df):
  # initialize the count variables
  prediction_t = 0
  prediction_h = 0
  ground_t = 0
  ground_h = 0
  SANITY_CHECK = False
  pred_df = []
  ground_df = []
  # weigh an image proportional to how much it contributes to the overall count.
  pred_weights = []
  ground_weights = []
  current_genotype_dir = COLLABORATION_DIR + row['Drive Path']
  print("Current Genotype: " + row['Genotype'])
  image_paths = get_image_paths(current_genotype_dir)
  GB = 'Gb' in row['Genotype']
  eval_column_names = ["Genotype", "Image Name", "Human Total", "ML Total", "Human # T", "ML # T", "Human # H", "ML # H", "Human % T", "ML % T"]
  eval_df = pd.DataFrame(columns = eval_column_names)
  # for all images of that genotype
  if KEYENCE_RATIO:
    for image_path in image_paths: 
        im = cv2.imread(image_path)
        im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
        im = np.stack((im,)*3, axis=-1)
        outputs = predictor(im)
        # https://github.com/facebookresearch/detectron2/issues/147
        prediction_list = outputs['instances'].pred_classes.flatten().tolist()
        prediction_counter = Counter(prediction_list)
        prediction_h += prediction_counter[1]
        prediction_t += prediction_counter[2]
        image_name = image_path.rsplit('/', 1)[-1]
        image_name = image_name[:-4]
        print("Image Name:" + image_name)
        current_h = 0
        current_t = 0
        ground_total = 30
        # if GB, we know that the total T count is 30.
        if not GB:
          image_row = greenhouse_df[greenhouse_df['Image Name'].str.contains(image_name)]
          image_row = image_row.iloc[0]
          current_h = int(image_row['#H'])
          current_t = int(image_row['#T'])
          assert int(image_row['Total']) == int(image_row['#H'])+int(image_row['#T'])
          ground_total = int(image_row['Total'])
        else:
          current_h = 0
          current_t = 30
          ground_total = 30

        ground_h += current_h
        ground_t += current_t
        current_pred_ratio_sum = prediction_counter[1] + prediction_counter[2]
        current_tapered_percentage = round(prediction_counter[2] / current_pred_ratio_sum, 2)
        pred_df.append(current_tapered_percentage)
        current_ground_ratio_sum = ground_total
        current_ground_t_percentage = round(current_t / current_ground_ratio_sum, 2)
        if not GB:
          assert "{:.1f}".format(float(image_row['%T'])/100) == "{:.1f}".format(float(current_ground_t_percentage))
        ground_df.append(current_ground_t_percentage)
        # add the count to the weights list
        pred_weights.append(prediction_counter[2])
        ground_weights.append(current_t)

        if SANITY_CHECK:
          v = Visualizer(im[:, :, ::-1],
                        metadata=val_metadata, scale = 0.5
          )
          print("### Predictions for {} ###".format(image_name))
          v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
          cv2_imshow(v.get_image()[:, :, ::-1])
          print("Prediction H Count: " + str(prediction_counter[1]))
          print("Prediction T Count: " + str(prediction_counter[2]))
          print("Human Label H Count: " + str(current_h))
          print("Human Label T Count: " + str(current_t))

        eval_data = {"Genotype": [row['Genotype']],
                     "Image Name": [image_name],
                     "Human Total": [ground_total],
                     "ML Total": [current_pred_ratio_sum],
                     "Human # T":[current_t],
                     "ML # T":[prediction_counter[2]],
                     "Human # H":[current_h],
                     "ML # H":[prediction_counter[1]],
                     "Human % T":[current_ground_t_percentage],
                     "ML % T":[current_tapered_percentage]}
        eval_data = pd.DataFrame(eval_data)
        eval_frames = [eval_df, eval_data]
        eval_df = pd.concat(eval_frames)
        display(eval_df)

    prediction_counter = [0,prediction_h, prediction_t]
    ground_truth_counter = [0,ground_h, ground_t]
    groundTruth = [ground_truth_counter[2], ground_truth_counter[1]]
    predictions = [prediction_counter[2], prediction_counter[1]]
    gt_ratio_sum = ground_truth_counter[2] + ground_truth_counter[1]
    gt_ratio_t = ground_truth_counter[2] / gt_ratio_sum   
    gt_ratio_h = ground_truth_counter[1] / gt_ratio_sum
    pred_ratio_sum = prediction_counter[2] + prediction_counter[1]
    pred_ratio_t = prediction_counter[2] / pred_ratio_sum 
    pred_ratio_h = prediction_counter[1] / pred_ratio_sum
    print("GT Total: ".format(gt_ratio_sum))
    print("Pred Total: ".format(pred_ratio_sum))
    gt_ratio_t = str(round(gt_ratio_t * 100, 2))
    gt_ratio_h = str(round(gt_ratio_h * 100, 2))
    pred_ratio_t = str(round(pred_ratio_t * 100, 2))
    pred_ratio_h = str(round(pred_ratio_h * 100, 2))
    ratios = [(gt_ratio_t, gt_ratio_h), (pred_ratio_t, pred_ratio_h)]
    assert prediction_h == prediction_counter[1]
    assert prediction_t == prediction_counter[2]
    assert len(pred_weights) == len(ground_weights) == len(image_paths)
    assert sum(ground_weights) == ground_t
    assert sum(pred_weights) == prediction_t
    assert "{}".format(round(float(row['%H']))) == "{}".format(round(float(gt_ratio_h)))
    assert "{}".format(round(float(row['%T']))) == "{}".format(round(float(gt_ratio_t)))
    pred_df = pd.DataFrame(pred_df,columns=['Percent Tapered'])
    ground_df = pd.DataFrame(ground_df,columns=['Percent Tapered'])
    pred_df_list = pred_df['Percent Tapered'].to_list()
    ground_df_list = ground_df['Percent Tapered'].to_list()
    print("Image Paths:")
    print(image_paths)
    print("prediction T count per image:")
    print(pred_weights)
    print("human label T count per image:")
    print(ground_weights)
    pred_weights = [round(current_count / prediction_t, 2) for current_count in pred_weights]
    ground_weights = [round(current_count / ground_t, 2) for current_count in ground_weights]
    assert sum(ground_weights) > 0.95
    assert sum(pred_weights) > 0.95
    print("prediction weights per image:")
    print(pred_weights)
    print("human label weights per image:")
    print(ground_weights)
    gt_percent_t = round(float(gt_ratio_t) / 100, 2)
    gt_percent_h = round(float(gt_ratio_h) / 100, 2)
    pred_percent_t = round(float(pred_ratio_t) / 100, 2)
    pred_percent_h = round(float(pred_ratio_h) / 100, 2)
    print("Ground Truth T-H ratio: {} - {}".format(gt_percent_t, gt_percent_h))
    print("Prediction T-H ratio: {} - {}".format(pred_percent_t, pred_percent_h))
    # compare human vs ml in barcharts per genotype
    if DISPLAY_BARCHARTS:
      gt_tapered = [groundTruth[0], "Tapered", "Ground Truth"]
      gt_hemispherical = [groundTruth[1], "Hemispherical", "Ground Truth"]
      pred_tapered = [predictions[0], "Tapered", "Prediction"]
      pred_hemispherical = [predictions[1], "Hemispherical", "Prediction"]
      df = pd.DataFrame([gt_tapered, gt_hemispherical, pred_tapered, pred_hemispherical],columns=('Count', 'Class', 'Type'))
      # citation:
      # https://stackoverflow.com/questions/55586912/seaborn-catplot-set-values-over-the-bars
      seaborn_plot = sns.catplot(x='Type', y='Count', hue='Class', data=df, kind='bar', palette=sns.color_palette("Pastel1"), color=123, saturation=1)
      seaborn_plot.fig.set_size_inches(10,8)
      seaborn_plot.fig.subplots_adjust(top=0.81,right=0.86) 
      ax = seaborn_plot.facet_axis(0,0)
      count = 0
      percentages = [gt_ratio_t, pred_ratio_t, gt_ratio_h, pred_ratio_h]
      for p in ax.patches:
          height = int(p.get_height())
          ax.text(p.get_x() + 0.05, 
                  p.get_height() * 1.02, 
                '{} ({}%)'.format(height, percentages[count]),  
                  color='black', 
                  rotation='horizontal',
                  size='large')
          count +=1
      # citation
      # https://stackoverflow.com/questions/29813694/how-to-add-a-title-to-seaborn-facet-plot
      seaborn_plot.fig.subplots_adjust(top=0.9)
      seaborn_plot.fig.suptitle("Ground Truth vs. Predictions with {}-Trained Model on {} Image {} Holdout Set".format(train_set_name, len(dataset_dicts), test_set_name))
    # compare T distributions across images for human and ML output
    if DISPLAY_PERCENTAGES:
      # https://www.geeksforgeeks.org/plotting-histogram-in-python-using-matplotlib/
      fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(16,8))
      axes = plt.gca()
      axs = [ax1, ax2]
      dfs = [ground_df_list, pred_df_list]
      axs[0].set_title("{}: GT % T on {} Images".format(row['Genotype'],len(image_paths)))
      axs[1].set_title("{}: Pred % T on {} Images".format(row['Genotype'],len(image_paths)))
      (ground_hist, bins_gt, patches) = axs[0].hist(dfs[0], bins=10, range=[0, 1], facecolor='#2a4d69', align='mid', weights = ground_weights)
      first_nonzero_index = next((i for i, x in enumerate(ground_hist) if x), None) # x!= 0 for strict match
      if first_nonzero_index == 0:
        first_nonzero_index == first_nonzero_index
      else:
        first_nonzero_index -= 1
      handles, _ = axs[0].get_legend_handles_labels()
      handles.append(axs[0].axvline(gt_percent_t, color='#fe9c8f', linestyle='dashed', linewidth=1))
      axs[0].legend(loc='upper left',handles = handles, labels = ['Total GT % T'])
      (pred_hist, bins_pred, patches) = axs[1].hist(dfs[1], bins=10, range=[0, 1], facecolor='#854442', align='mid', weights = pred_weights)
      first_nonzero_index_pred = next((i for i, x in enumerate(pred_hist) if x), None) # x!= 0 for strict match
      if first_nonzero_index_pred == 0:
        first_nonzero_index_pred == first_nonzero_index_pred
      else:
        first_nonzero_index_pred -= 1
      largest_x = max(bins_gt[first_nonzero_index], bins_pred[first_nonzero_index_pred])
      largest_y = max(max(ground_hist), max(pred_hist))
      largest_y = float(ceil(largest_y*10) / 10)
      ax1.set_xlim([largest_x,1])
      ax2.set_xlim([largest_x,1])
      ax1.set_ylim([0,largest_y])
      ax2.set_ylim([0,largest_y])
      ax1.set_xlabel("Tapered Percentage")
      ax2.set_xlabel("Tapered Percentage")
      ax1.set_ylabel("Proportion")
      ax2.set_ylabel("Proportion")
      fig.tight_layout(pad=3.0)
      handles, _ = axs[1].get_legend_handles_labels()
      # https://stackoverflow.com/questions/24988448/how-to-draw-vertical-lines-on-a-given-plot
      handles.append(axs[1].axvline(pred_percent_t, color='#35a79c', linestyle='dashed', linewidth=1))
      axs[1].legend(loc='upper left',handles = handles, labels = ['Total Pred % T'])
      pred_std_dev = round(pred_df.std(numeric_only=True)['Percent Tapered'], 2)
      ground_std_dev = round(ground_df.std(numeric_only=True)['Percent Tapered'], 2)
      # metric of how different two distributions are:
      # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wasserstein_distance.html
      emd = round(wasserstein_distance(ground_hist, pred_hist),2)
      print("Prediction Std Dev: {}".format(pred_std_dev))
      print("Ground Std Dev: {}".format(ground_std_dev))
      print("Earth Mover's Distance: {}".format(emd))
      human_ratio = gt_ratio_t + '-' + gt_ratio_h
      ML_ratio = pred_ratio_t + '-' + pred_ratio_h
      ratio_diff = abs(float(gt_ratio_t) - float(pred_ratio_t))
      column_names = ["Genotype", "Human T:H Ratio", "Human Total Fibers", "ML T:H Ratio", "ML Total Fibers","Ratio Diff", "Human Std Dev", "ML Std Dev"]
      data = {"Genotype": [row['Genotype']], "Human T:H Ratio":[human_ratio], "Human Total Fibers": [gt_ratio_sum],
              "ML T:H Ratio": [ML_ratio], "ML Total Fibers": [pred_ratio_sum], "Ratio Diff": [ratio_diff],
              "Human Std Dev":[round(ground_std_dev*100, 1)], "ML Std Dev":[round(pred_std_dev*100, 1)]}
      data_hist = {"Genotype": [row['Genotype']], "Human Total %T Mean": [gt_percent_t],"ML Total %T Mean": [pred_percent_t], "Human Hist":[list(ground_hist)], "ML Hist": [list(pred_hist)], "Bins": [10], "Range":[[0, 1]]}
      # Convert the dictionary into DataFrame
      geno_df = pd.DataFrame(data)
      data_hist = pd.DataFrame(data_hist)

    return ratios, geno_df, data_hist, eval_df

### Calculate ratio differences between humans and ML

In [ ]:
def calculate_ratio_difference(groundTruth, prediction):
  t_diff = abs(float(groundTruth[0]) - float(prediction[0])) 
  h_diff = abs(float(groundTruth[1]) - float(prediction[1]))
  return t_diff + h_diff

def genotype_ratio_difference(predictor, row, val_metadata, train_set_name, greenhouse_df):
  ratios, geno_df,data_hist, eval_df = get_genotype_ratios(predictor, row, val_metadata, train_set_name, greenhouse_df)
  return calculate_ratio_difference(ratios[0], ratios[1]), geno_df,data_hist, eval_df

# Final Evaluation

In [ ]:
def genotype_evaluation(cfg, best_model_path_name, holdout_set, th_df, output_dir, train_set_name, greenhouse_df):
  current_model_ratio_difference = 10000
  # instantiate the model
  cfg, predictor, dataset_dicts, val_metadata = instantiate_model(cfg, best_model_path_name, dataset_name, output_dir)
  display(th_df)
  ratio_list = []
  # create dataset file names.
  MODEL_PATH_NAME = best_model_path_name.split(".pth",1)[0]
  DATASET_FILENAME = "/greenhouse_total_results_{}_{}.csv".format(today,MODEL_PATH_NAME)
  SUMMARY_GREENHOUSE_FILENAME = HOME  + DATASET_FILENAME
  HIST_DATASET_FILENAME = "/hist_greenhouse_{}_{}.csv".format(today,MODEL_PATH_NAME)
  HIST_GREENHOUSE_NAME = HOME + HIST_DATASET_FILENAME
  EVAL_DATASET_FILENAME = "/greenhouse_per_image_results_{}_{}.csv".format(today,MODEL_PATH_NAME)
  INDIVIDUAL_DP90_FILENAME = HOME + EVAL_DATASET_FILENAME
  print(SUMMARY_GREENHOUSE_FILENAME)
  print(HIST_GREENHOUSE_NAME)
  print(INDIVIDUAL_DP90_FILENAME)
  column_names = ["Genotype", "Human T:H Ratio", "Human Total Fibers", "ML T:H Ratio", "ML Total Fibers","Ratio Diff", "Human Std Dev", "ML Std Dev"]
  geno_df = pd.DataFrame(columns = column_names)
  hist_column_names = ["Genotype","Human Total %T Mean","ML Total %T Mean", "Human Hist", "ML Hist", "Bins","Range"]
  hist_df = pd.DataFrame(columns = hist_column_names)
  eval_column_names = ["Genotype", "Image Name", "Human Total", "ML Total", "Human # T", "ML # T", "Human # H", "ML # H", "Human % T", "ML % T"]
  eval_df = pd.DataFrame(columns = eval_column_names)

  if CALCULATE_RATIOS:
    th_df = th_df.reset_index()  # make sure indexes pair with number of rows
    for index, row in th_df.iterrows():
      current_model_ratio_difference, genotype_row, data_hist, eval_data = genotype_ratio_difference(predictor, row, val_metadata, train_set_name, greenhouse_df)
      ratio_list.append(current_model_ratio_difference)
      frames = [geno_df, genotype_row]
      geno_df = pd.concat(frames)
      display(geno_df)
      hist_frames = [hist_df, data_hist]
      hist_df = pd.concat(hist_frames)
      display(hist_df)
      eval_frames = [eval_df, eval_data]
      eval_df = pd.concat(eval_frames)
      display(eval_df)
  geno_df.to_csv(SUMMARY_GREENHOUSE_FILENAME)  
  hist_df.to_csv(HIST_GREENHOUSE_NAME)  
  eval_df.to_csv(INDIVIDUAL_DP90_FILENAME)  
  return ratio_list

# grid search for best model and best confidence threshold.
def best_genotype_evaluation(cfg, model_paths, holdout_set, th_df, output_dir, train_set_name, thresholds, greenhouse_df):
  best_model_difference = 10000
  best_model_path_name = ''
  best_threshold = 1
  for model_path in model_paths:
    if model_path is not '':
      for threshold in thresholds:
        TEST_THRESHOLD = threshold
        print("Current Model: " + model_path)
        print("Current Output Dir: " + output_dir)
        print("Current threshold: {}".format(TEST_THRESHOLD))
        ratio_list = genotype_evaluation(cfg, model_path, holdout_set, th_df, output_dir, train_set_name, greenhouse_df)

        if sum(ratio_list) < best_model_difference:
          best_model_difference = sum(ratio_list)
          best_model_path_name = model_path
          best_threshold = threshold
          print("Best Model: " + best_model_path_name)
          print("Best Model Ratio Difference: {}".format(best_model_difference))
          print("Best Threshold: {}".format(best_threshold))
  return best_model_path_name, best_model_difference, best_threshold

# Final Greenhouse Evaluation

In [ ]:
def genotype_final(cfg, best_model_path_name, th_df, output_dir, train_set_name, greenhouse_df):
   # this doesn't matter, I don't think.
  holdout_set = "test"
  genotype_evaluation(cfg, best_model_path_name, holdout_set, th_df, output_dir, train_set_name, greenhouse_df)

output_dirs = [OUTPUT_DIR]
for output_dir in output_dirs:
  DISPLAY_BARCHARTS = False
  CALCULATE_RATIOS = True
  EVALUATE_MODEL = False
  dataset_name = "test"
  train_set_name = "Keyence"
  test_set_name = "Keyence"
  KEYENCE_RATIO = True
  output_dir = OUTPUT_DIR
  DISPLAY_BARCHARTS = False
  CALCULATE_RATIOS = True
  EVALUATE_MODEL = False
  genotype_final(cfg, best_model_path_name, th_df, output_dir, train_set_name, greenhouse_df)